In [162]:
import pandas as pd
import os
os.chdir(r"/Users/apple/Documents/Credit_Card_Churn_Model_Local/credit-card-churn/Input")

In [163]:
#Initialising Dataset

data = pd.read_csv("BankChurners.csv") 
data = data.drop(data.columns[-2:], axis=1)  
data.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


In [164]:
#Removing CLIENTNUM
data.drop(['CLIENTNUM'], axis=1, inplace=True) 
data.shape

(10127, 20)

In [165]:
#Unique Values and its frequency in categorical columns
columns = ['Attrition_Flag','Education_Level','Marital_Status','Income_Category','Card_Category',]
for col in columns:
    print(data[col].value_counts())


Attrition_Flag
Existing Customer    8500
Attrited Customer    1627
Name: count, dtype: int64
Education_Level
Graduate         3128
High School      2013
Unknown          1519
Uneducated       1487
College          1013
Post-Graduate     516
Doctorate         451
Name: count, dtype: int64
Marital_Status
Married     4687
Single      3943
Unknown      749
Divorced     748
Name: count, dtype: int64
Income_Category
Less than $40K    3561
$40K - $60K       1790
$80K - $120K      1535
$60K - $80K       1402
Unknown           1112
$120K +            727
Name: count, dtype: int64
Card_Category
Blue        9436
Silver       555
Gold         116
Platinum      20
Name: count, dtype: int64


In [166]:
data['Attrition_Flag'] = data['Attrition_Flag'].map({ 
    'Attrited Customer': 1,
    'Existing Customer': 0})

In [167]:
data['Education_Level'] = data['Education_Level'].map({ 'Uneducated': 0,
    'High School': 1,
    'College': 2,
    'Graduate': 3,
    'Post-Graduate': 4,
    'Doctorate': 5,
    'Unknown': -1})

In [168]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import numpy as np
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
data['Gender'] = np.array(ct.fit_transform(data['Gender'].values.reshape(-1, 1)))


In [169]:
data['Income_Category'] = data['Income_Category'].map({ 
    'Less than $40K': 1,
    '$40K - $60K': 2,
    '$60K - $80K': 3,
    '$80K - $120K': 4,
    '$120K +': 5,
    'Unknown': -1})

In [170]:
data['Card_Category'] = data['Card_Category'].map({ 
    'Blue': 1,
    'Silver': 2,
    'Gold': 3,
    'Platinum': 4,
    'Unknown': -1})

In [171]:
#Usinng pandas converting Categorical variable to dummies
dummies = pd.get_dummies(data['Marital_Status'], drop_first=True)
dummies = dummies.astype(int)
dummies
# Concatenate the dummies with the original dataframe   
data = pd.concat([data, dummies], axis= 1)
# Drop the original 'Marital_Status' column
data.drop(['Marital_Status'], axis=1, inplace=True)


In [172]:
import pandas as pd
import numpy as np

# Binning Age
age_bins = [0, 30, 40, 50, 60, np.inf]
age_labels = ['Under 30', '30-40', '40-50', '50-60', '60+']
data['Customer_Age_Binned'] = pd.cut(data['Customer_Age'], bins=age_bins, labels=age_labels)

# Creating dummy variables for Age_Binned
age_dummies = pd.get_dummies(data['Customer_Age_Binned'], prefix='Age')

# Adding the dummy variables to the original DataFrame
data = pd.concat([data, age_dummies], axis=1)

In [173]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   Attrition_Flag            10127 non-null  int64   
 1   Customer_Age              10127 non-null  int64   
 2   Gender                    10127 non-null  float64 
 3   Dependent_count           10127 non-null  int64   
 4   Education_Level           10127 non-null  int64   
 5   Income_Category           10127 non-null  int64   
 6   Card_Category             10127 non-null  int64   
 7   Months_on_book            10127 non-null  int64   
 8   Total_Relationship_Count  10127 non-null  int64   
 9   Months_Inactive_12_mon    10127 non-null  int64   
 10  Contacts_Count_12_mon     10127 non-null  int64   
 11  Credit_Limit              10127 non-null  float64 
 12  Total_Revolving_Bal       10127 non-null  int64   
 13  Avg_Open_To_Buy           10127 non-null  floa

In [174]:
import pandas as pd
import numpy as np


# Binning Total_Trans_Amt
trans_amt_bins = [0, 1000, 5000, 10000, np.inf]
trans_amt_labels = ['Low', 'Medium', 'High', 'Very High']
data['Total_Trans_Amt_Binned'] = pd.cut(data['Total_Trans_Amt'], bins=trans_amt_bins, labels=trans_amt_labels)

# Binning Avg_Open_To_Buy
open_to_buy_bins = [0, 5000, 15000, 30000, np.inf]
open_to_buy_labels = ['Low', 'Medium', 'High', 'Very High']
data['Avg_Open_To_Buy_Binned'] = pd.cut(data['Avg_Open_To_Buy'], bins=open_to_buy_bins, labels=open_to_buy_labels)

# Binning Total_Revolving_Bal
revolving_bal_bins = [0, 500, 1500, 3000, np.inf]
revolving_bal_labels = ['Low', 'Medium', 'High', 'Very High']
data['Total_Revolving_Bal_Binned'] = pd.cut(data['Total_Revolving_Bal'], bins=revolving_bal_bins, labels=revolving_bal_labels)

# Creating dummy variables for all binned columns
dummies = pd.get_dummies(data[['Total_Trans_Amt_Binned', 'Avg_Open_To_Buy_Binned', 
                               'Total_Revolving_Bal_Binned']])

# Adding the dummy variables to the original DataFrame
data = pd.concat([data, dummies], axis=1)



In [175]:
data.drop(['Customer_Age', 'Customer_Age_Binned', 'Age_60+', 
           'Total_Trans_Amt', 'Total_Trans_Amt_Binned','Total_Trans_Amt_Binned_Very High',
           'Avg_Open_To_Buy', 'Avg_Open_To_Buy_Binned', 'Avg_Open_To_Buy_Binned_Very High',
           'Total_Revolving_Bal','Total_Revolving_Bal_Binned','Total_Revolving_Bal_Binned_Very High',], axis=1, inplace=True)

In [178]:
import os

# Get the directory of the input file
input_file = 'cleaned_credit_card_churn.csv'
output_file = 'cleaned_credit_card_churn_numeric.csv'
directory = os.path.dirname(input_file)

# Save the numeric data to the new file in the same directory
data.to_csv(os.path.join(directory, output_file), index=False)

In [180]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 31 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Attrition_Flag                     10127 non-null  int64  
 1   Gender                             10127 non-null  float64
 2   Dependent_count                    10127 non-null  int64  
 3   Education_Level                    10127 non-null  int64  
 4   Income_Category                    10127 non-null  int64  
 5   Card_Category                      10127 non-null  int64  
 6   Months_on_book                     10127 non-null  int64  
 7   Total_Relationship_Count           10127 non-null  int64  
 8   Months_Inactive_12_mon             10127 non-null  int64  
 9   Contacts_Count_12_mon              10127 non-null  int64  
 10  Credit_Limit                       10127 non-null  float64
 11  Total_Amt_Chng_Q4_Q1               10127 non-null  flo